# Basic scVI Tutorial

### Loading Config

In [1]:
%matplotlib inline
import os
os.chdir("../..")
import json
with open('basic_tutorial.config.json') as f:
    config = json.load(f)
print(config)

n_epochs_all = config['n_epochs'] if 'n_epochs' in config else None
save_path = config['save_path'] if 'save_path' in config else 'data/'
n_samples_tsne = config['n_samples_tsne'] if 'n_samples_tsne' in config else None
n_samples_posterior_density = config['n_samples_posterior_density'] if 'n_samples_posterior_density' in config else None
train_size = config['train_size'] if 'train_size' in config else None
M_sampling_all = config['M_sampling'] if 'M_sampling' in config else None
M_permutation_all = config['M_permutation'] if 'M_permutation' in config else None
rate = config['rate'] if 'rate' in config else None

FileNotFoundError: [Errno 2] No such file or directory: 'basic_tutorial.config.json'

In [5]:
import os

import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


from scvi.dataset import CortexDataset, RetinaDataset

from scvi.models import *
from scvi.inference import UnsupervisedTrainer

## Loading data

Here we load the CORTEX dataset described in

* Zeisel, Amit, et al. "Cell types in the mouse cortex and hippocampus revealed by single-cell RNA-seq." Science 347.6226 (2015): 1138-1142. 

Please see our data loading Jupyter notebook for more examples of data loading -- scVI has many "built-in" datasets, as well as support for loading arbitrary .csv, .loom, and .h5ad (AnnData) files.

In [3]:
gene_dataset = CortexDataset(save_path=save_path)

File data/expression.bin already downloaded
Preprocessing Cortex data
Finished preprocessing Cortex data
labels: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6

## Training

* __n_epochs__: Maximum number of epochs to train the model. If the likelihood change is small than a set threshold training will stop automatically. 
* __lr__: learning rate. Set to 0.001 here. 
* __use_batches__: If the value of true than batch information is used in the training. Here it is set to false because the cortex data only contains one batch. 
* __use_cuda__: Set to true to use CUDA. 


In [4]:
n_epochs=400 if n_epochs_all is None else n_epochs_all
lr=1e-3
use_batches=False
use_cuda=True

**Train the model and output model likelihood every 5 epochs**

In [1]:
vae = VAE(gene_dataset.nb_genes, n_batch=gene_dataset.n_batches * use_batches)
trainer = UnsupervisedTrainer(vae,
                              gene_dataset,
                              train_size=0.75,
                              use_cuda=use_cuda,
                              frequency=5)
trainer.train(n_epochs=n_epochs, lr=lr)

NameError: name 'VAE' is not defined

** Plotting the likelihood change across the 500 epochs of training: blue for training error and orange for testing error. **

In [6]:
ll_train_set = trainer.history["ll_train_set"]
ll_test_set = trainer.history["ll_test_set"]
x = np.linspace(0,500,(len(ll_train_set)))
plt.plot(x, ll_train_set)
plt.plot(x, ll_test_set)
plt.ylim(1150,1600)
plt.show()

## Visualizing the latent space

In [7]:
trainer.train_set.show_t_sne(n_samples=n_samples_tsne, color_by='labels', save_name="basic_t")

plt_labels: ['astrocytes_ependymal' 'endothelial-mural' 'interneurons' 'microglia'
 'oligodendrocytes' 'pyramidal CA1' 'pyramidal SS']


## Obtaining the latent space for the whole dataset

This will return in a np.array the mean of the posterior distribution p(z|x) where x is the data whose latent representation we want. In the np.array giving the latent space, the cells are in the same order as in the original dataset matrix. As you can see in the imputation section, the get_all_latent_and_imputed_values method provides more information than just the latent space, it is a dictionary that contains the latent space, the imputed values, the labels and batch indices of the dataset

In [8]:
latent = trainer.get_all_latent_and_imputed_values()["latent"]
#in order to save the latent space you can either use numpy.save 
#or if you want it as a .csv file you can add arguments to the method get_dataset_information which will do it
latent = trainer.get_all_latent_and_imputed_values(save_latent=True, file_name_latent="Cortex_dataset_latent_space")["latent"]


## Imputation


The ability to impute missing values is useful in practical applications in addition to providing an assay for generalization performance. In the following analysis, we benchmark scVI against BISCUIT, ZINB-WaVE and ZIFA, as well as MAGIC, which provides imputation without explicit statistical modeling. To evaluate these methods on a given dataset, we generated a **corrupted training set**, and then fitted the perturbed dataset with each of the benchmark methods and evaluate them by comparing the imputed values to the original ones (Methods 4.7). Overall, we observe that the imputation accuracy of scVI is higher or comparable (less than one transcript for median error) across all datasets

#### Corrupting the datasets for imputation benchmarking. 

Two different approaches to measure the robustness of algorithms to noise in the data: 

- **Uniform zero introduction**: select randomly a rate r% of the non-zero entries and multiply the entry n with a Ber(0.9) random variable.
- **Binomial data corruption**: select a rate r% of the matrix and replace an entry n by a Bin(n, 0.2) random variable.

By default, the rate r is set a 0.1

#### Accuracy of imputing missing data 

As imputation tantamount to replace missing data by its mean conditioned on being observed, we use the median L1 distance between the original dataset and the imputed values for corrupted entries only.
Parameters:
* The rate of simulated dropout is defined by __rate__, here set ot 0.1 

In [9]:
n_epochs = 400 if n_epochs_all is None else n_epochs_all
vae = VAE(gene_dataset.nb_genes, n_batch=gene_dataset.n_batches * use_batches)
trainer = UnsupervisedTrainer(vae,
                              gene_dataset,
                              train_size=0.75 if train_size is None else train_size,
                              use_cuda=use_cuda)

trainer.corrupt_posteriors(rate=0.1, corruption="uniform")
trainer.train(n_epochs)
trainer.uncorrupt_posteriors()

original_list, imputed_list = trainer.train_set.imputation_benchmark(verbose=True, n_samples=1)

training: 100%|██████████| 400/400 [05:43<00:00,  1.17it/s]

Median of Median: 2.4180
Mean of Median for each cell: 3.5523


In [10]:
# Median of medians for all distances
imputation_errors = np.abs(np.concatenate(original_list) - np.concatenate(imputed_list))
median_imputation_score = np.median(imputation_errors)
print(median_imputation_score)

2.418014


We then plot the distribution of absolute errors between the imputed value and the true value at the dropout positions. 

Note: The imputed value __px_rate__ is the rate parameter (expected value) of the Zero-Inflated Negative Binomial (ZINB) distribution.

In [11]:
plt.hist(np.log10(imputation_errors))

(array([1.000e+00, 3.000e+00, 6.000e+00, 2.900e+01, 2.570e+02, 1.891e+03,
        5.131e+03, 1.474e+03, 7.300e+01, 9.000e+00]),
 array([-5.37962198, -4.48792462, -3.59622726, -2.70452991, -1.81283255,
        -0.92113519, -0.02943783,  0.86225953,  1.75395689,  2.64565425,
         3.53735161]),
 <a list of 10 Patch objects>)

### Imputing without artificial corruption

Through the imputation benchmark above, we have assessed the strength of scVI for this task. 

To actually perform the imputation on a model trained on regular data, we might use the `.imputation` method. As stochasticity of the training requires iteration on permuted samples of the data, the output result is in random order. 

To get an ordered output result, we might use `.sequential` posterior's method which return another instance of posterior (with shallow copy of all its object references), but where the iteration is in the same ordered as its  indices attribute.


In [12]:
imputed_values_unordered = trainer.train_set.imputation()

indices = trainer.train_set.indices # The indices in the order that they will be processed with .sequential()
#These indices correspond to the indices of the cells in the input counts matrix vae.gene_dataset.X
imputed_values_indices = trainer.train_set.sequential().imputation() # imputation result ordered along indices

## Obtaining the imputed values for the whole dataset after the training


In [13]:
imputed_values = trainer.get_all_latent_and_imputed_values()["imputed_values"]
#in order to save it you can either use numpy.save 
#or if you want it as a .csv file you can add arguments to the method get_dataset_information which will do it
imputed_values = trainer.get_all_latent_and_imputed_values(save_imputed=True, file_name_imputation="Cortex_dataset_imputed_values")["imputed_values"]
#there is another optional argument: the boolean save_shape_genes_by_cells 
#so you can choose according to which format you prefer to save the expression matrix

## Differential Expression
From the trained VAE model we can sample the gene expression rate for each gene in each cell. For the two populations of interest, we can then randomly sample pairs of cells, one from each population to compare their expression rate for a gene. The degree of differential expression is measured by __logit(p/(1-p))__ where __p__ is the probability of a cell from population A having a higher expression than a cell from population B. We can form the null distribution of the DE values by sampling pairs randomly from the combined population.

The following example is implemented for the cortext dataset, vary __cell_types__ and __genes_of_interest__ for other datasets. 

**1. Set population A and population B for comparison**

In [14]:
cell_types = gene_dataset.cell_types
print(gene_dataset.cell_types)
# oligodendrocytes (#4) VS pyramidal CA1 (#5)
couple_celltypes = (4, 5)  # the couple types on which to study DE

print("\nDifferential Expression A/B for cell types\nA: %s\nB: %s\n" %
      tuple((cell_types[couple_celltypes[i]] for i in [0, 1])))

['astrocytes_ependymal' 'endothelial-mural' 'interneurons' 'microglia'
 'oligodendrocytes' 'pyramidal CA1' 'pyramidal SS']

Differential Expression A/B for cell types
A: oligodendrocytes
B: pyramidal CA1



**2. Define parameters**
* __M_sampling__: the number of times to sample __px_scales__ from the vae model for each gene in each cell.
* __M_permutation__: Number of pairs sampled from the px_scales values for comparison.

In [15]:
M_sampling = 100 if M_sampling_all is None else M_sampling_all

M_permutation = 100000  if M_permutation_all is None else M_permutation_all
permutation = False

**3. Sample from the gene expression level from all cells**
Note: The expectation of the ZINB distribution __px_rate ~ library_size * px_scale__, so __px_scale__ could be understood as the mean gene expression level of each cell after adjusting for the library size factor. 

In [16]:
px_scale, all_labels = trainer.train_set.differential_expression_stats(M_sampling=M_sampling)

**4. Extract the sampled gene expression level for the two populations of interest, and create indexes for the samples**

In [17]:
sample_rate_a = px_scale[(all_labels == couple_celltypes[0]).ravel()].reshape(-1, px_scale.shape[1])
sample_rate_b = px_scale[(all_labels == couple_celltypes[1]).ravel()].reshape(-1, px_scale.shape[1])

list_1 = list(np.arange(sample_rate_a.shape[0]))
list_2 = list(sample_rate_a.shape[0] + np.arange(sample_rate_b.shape[0]))

samples = np.vstack((sample_rate_a, sample_rate_b))

**5. Compute whether a gene is differentially expressed by computing pairs of cells from population A and population B**

In [18]:
u, v = np.random.choice(list_1, size=M_permutation), np.random.choice(list_2, size=M_permutation)
first_set = samples[u]
second_set = samples[v]
res1 = np.mean(first_set >= second_set, 0)
res1 = np.log(res1 + 1e-8) - np.log(1 - res1 + 1e-8)

**6. Obtaining the null value by comparing pairs sampled from the combined population**

In [19]:
u, v = (np.random.choice(list_1 + list_2, size=M_permutation),
    np.random.choice(list_1 + list_2, size=M_permutation))
first_set = samples[u]
second_set = samples[v]
res2 = np.mean(first_set >= second_set, 0)
res2 = np.log(res2 + 1e-8) - np.log(1 - res2 + 1e-8)

**7. Print out the differential expression value from both the true comparison and the permuted comparison**

In [20]:
genes_of_interest = ["Thy1", "Mbp"]
gene_names = gene_dataset.gene_names
result = [(gene_name, res1[np.where(gene_names == gene_name.upper())[0]][0],res2[np.where(gene_names == gene_name.upper())[0]][0]) for gene_name in genes_of_interest]
print('\n'.join([gene_name + " : " + str(r1) + " , "+ str(r2) for (gene_name, r1,r2) in result]))

Thy1 : -5.74453613852952 , 0.0010800000833760537
Mbp : 5.447166996632846 , -0.0007200000167042475


**1-7. Step one through seven obtained with a single line **

Give as argument the two cell types and genes of interest and it will return the list of the bayes factor score for each gene in the same order specified.

In [21]:
bayes_factors_list = trainer.train_set.differential_expression_score(
    'oligodendrocytes', 'pyramidal CA1',
    M_sampling=M_sampling,
    M_permutation=M_permutation,
    genes= ["THY1", "MBP"]
)

**8. Plot the null distribution of the DE values**

In [22]:
plt.hist(res2)

(array([  3.,  19.,  38.,  64.,  98., 129., 112.,  64.,  24.,   7.]),
 array([-0.01632036, -0.0132643 , -0.01020824, -0.00715218, -0.00409612,
        -0.00104006,  0.002016  ,  0.00507206,  0.00812812,  0.01118418,
         0.01424024]),
 <a list of 10 Patch objects>)

**9. Visualize top 10 most expressed genes per cell types**

In [23]:
genes, expression = trainer.train_set.differential_expression_table(M_sampling=M_sampling)

plt.figure(figsize=(20, 20))
im = plt.imshow(expression, cmap='RdYlGn', interpolation='none', aspect='equal')
ax = plt.gca()
ax.set_xticks(np.arange(0, 7, 1))
ax.set_xticklabels(gene_dataset.cell_types, rotation='vertical')
ax.set_yticklabels(genes)
ax.set_yticks(np.arange(0, 70, 1))
ax.tick_params(labelsize=14)
plt.colorbar(shrink=0.2)

## Correction for batch effects

First we load the RETINA dataset that is described in

* Shekhar, Karthik, et al. "Comprehensive classification of retinal bipolar neurons by single-cell transcriptomics." Cell 166.5 (2016): 1308-1323.

In [24]:
gene_dataset = RetinaDataset(save_path=save_path)

File data/retina.loom already downloaded
Preprocessing dataset
Finished preprocessing dataset


In [ ]:
n_epochs=50 if n_epochs_all is None else n_epochs_all
lr=1e-3
use_batches=True
use_cuda=True

### Train the model and output model likelihood every 5 epochs
vae = VAE(gene_dataset.nb_genes, n_batch=gene_dataset.n_batches * use_batches)
trainer = UnsupervisedTrainer(vae, 
                              gene_dataset, 
                              train_size=0.9, 
                              use_cuda=use_cuda,
                              frequency=5)
trainer.train(n_epochs=n_epochs, lr=lr)

training:  84%|████████▍ | 42/50 [2:18:09<26:18, 197.37s/it]  

In [ ]:
# Plotting the likelihood change across the 50 epochs of training: blue for training error and orange for testing error. 

ll_train = trainer.history["ll_train_set"]
ll_test = trainer.history["ll_test_set"]
x = np.linspace(0,50,(len(ll_train)))
plt.plot(x, ll_train)
plt.plot(x, ll_test)
plt.ylim(min(ll_train)-50, 3500)
plt.show()

**Computing batch mixing**

In [ ]:
print("Entropy batch mixing :", trainer.train_set.entropy_batch_mixing())

**Coloring by batch and cell type**

In [ ]:
# obtaining latent space in the same order as the input data
trainer.train_set.show_t_sne(n_samples=n_samples_tsne, color_by='batches and labels')

In [ ]:
def allow_notebook_for_test():
    print("Testing the basic tutorial notebook")